<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-01 10:47:22
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -27.54 K (-0.2%)
Current PnL: -13.78 L (-9.6%)
CY Booked + Current PnL: -7.04 L (-4.9%)
-------------------
Total profit:  4.35 L
Total loss:  -18.13 L
-------------------
Total Booked + Current PnL: 19.94 L (17.11%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.95%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.20 L (53.04%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,83.33,74218.0,39769.0,178.0,-0.24,115.44,0.24,115.97,NaN,223.42,0.55,13.49,AR,ATH,ELECTRICAL
16,BLUSPRING,226.45,84.55,NaN,NaN,3.12,16263.0,-33330.0,2254.0,-2.21,-67.21,13.86,-62.66,NaN,-14.79,0.12,9.06,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42972.0,1717.0,9089.0,0.05,4.16,21.15,26.19,1.0,0.19,0.32,0.05,XY24,ATH,IT
3,ADANIPORTS,1103.69,1583.00,2.76,M-LC,29.17,103007.0,24645.0,9384.0,0.04,31.45,9.11,43.43,77.0,2.63,0.76,38.44,XY24,NTT,MISC
87,UJJIVANSFB,52.77,53.00,51.69,M-SC,88.54,131382.0,-11097.0,11719.0,-0.75,-7.79,8.92,0.44,243.0,-0.95,0.97,55.07,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,87.81,M-MC,36.46,225272.0,310.0,167355.0,-0.89,0.14,74.29,74.53,185.0,0.00,1.66,30.79,XY24,BTT,STEEL
12,BANKINDIA,116.91,190.00,-31.29,M-MC,19.79,181453.0,1645.0,110759.0,-0.49,0.92,61.04,62.52,171.0,0.01,1.33,35.31,XR,NTT,BANKS
55,LALPATHLAB,2804.19,3545.00,-33.29,H-SC,44.79,153510.0,-720.0,41463.0,-0.19,-0.47,27.01,26.42,125.0,-0.02,1.13,21.06,X40N,NTT,HEALTHCARE
60,NATIONALUM,189.63,247.44,-50.15,H-MC,64.58,100530.0,26.0,30611.0,-1.57,0.03,30.45,30.49,97.0,0.00,0.74,32.44,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,SIEMENS,4406.90,7969.85,-0.26,X-LC,51.04,84230.0,-25942.0,115016.0,3.62,-23.55,136.55,80.85,14.0,-0.23,0.62,24.06,AR,ATH,ELECTRICAL
5,APOLLOHOSP,6904.43,8285.00,-17.96,H-LC,77.08,111802.0,8236.0,12477.0,2.92,7.95,11.16,20.00,30.0,0.66,0.82,23.15,X40N,BTT,HEALTHCARE
54,KPIGREEN,497.21,732.16,29.50,H-SC,75.00,132464.0,7167.0,52045.0,2.14,5.72,39.29,47.25,139.0,0.14,0.97,65.72,MH,ATH,POWER
15,BLUESTARCO,1646.70,2337.55,-11.10,H-SC,78.12,166780.0,2110.0,66979.0,1.97,1.28,40.16,41.95,121.0,0.03,1.23,8.84,X40N,ATH,AC
90,VALIANTORG,512.64,838.00,-355.15,H-SC,6.25,108589.0,-24697.0,109295.0,1.93,-18.53,100.65,63.47,148.0,-0.23,0.80,82.02,XR,NTT,CHEMICALS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,DIXON,14367.00,18940.15,-36.69,H-MC,30.21,116496.0,1560.0,35030.0,-2.81,1.36,30.07,31.83,99.0,0.04,0.86,33.52,X40N,ATH,IT
4,ALKYLAMINE,2347.98,4546.37,6.93,X-SC,98.96,99524.0,-1439.0,95971.0,-2.26,-1.43,96.43,93.63,64.0,-0.01,0.73,48.24,SR,ATH,CHEMICALS
16,BLUSPRING,226.45,84.55,NaN,NaN,3.12,16263.0,-33330.0,2254.0,-2.21,-67.21,13.86,-62.66,NaN,-14.79,0.12,9.06,XY24,ATH,MISC
70,SATIN,199.77,274.00,-17.21,H-SC,80.21,163474.0,-32500.0,105326.0,-1.79,-16.58,64.43,37.16,130.0,-0.31,1.20,22.08,XY24,NTT,FINANCE
77,STARHEALTH,551.56,761.00,-7.87,M-SC,2.08,150790.0,-49426.0,125457.0,-1.73,-24.69,83.20,37.97,215.0,-0.39,1.11,21.07,XY24,NTT,INSURANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,51.69,M-SC,88.54,131382.0,-11097.0,11719.0,-0.75,-7.79,8.92,0.44,243.0,-0.95,0.97,55.07,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-215.40,H-SC,97.92,83265.0,-12051.0,12090.0,1.39,-12.64,14.52,0.04,154.0,-1.00,0.61,64.23,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-16.81,X-MC,26.04,87416.0,-15236.0,15228.0,-1.05,-14.84,17.42,-0.00,58.0,-1.00,0.64,32.88,OX40N,NTT,IT
46,INDIGOPNTS,1407.73,1408.0,122.10,H-SC,89.58,145750.0,-28809.0,28844.0,1.14,-16.50,19.79,0.02,127.0,-1.00,1.07,26.39,OX40N,NTT,PAINTS
53,KANSAINER,299.63,340.0,-67.65,H-SC,37.50,224820.0,-44847.0,81183.0,-0.93,-16.63,36.11,13.47,142.0,-0.55,1.65,13.71,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42972.0,1717.0,9089.0,0.05,4.16,21.15,26.19,1.0,0.19,0.32,0.05,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.42,X-LC,41.67,269186.0,-26748.0,120649.0,-0.31,-9.04,44.82,31.73,1.0,-0.22,1.98,7.74,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.62,X-LC,59.38,286722.0,25121.0,120509.0,-0.39,9.60,42.03,55.67,2.0,0.21,2.11,16.29,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.84,X-LC,21.88,257331.0,-13016.0,47632.0,0.13,-4.81,18.51,12.81,6.0,-0.27,1.89,7.62,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-9.55,X-LC,47.92,286732.0,21306.0,35612.0,-0.59,8.03,12.42,21.44,9.0,0.60,2.11,14.72,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,-1.39,M-LC,27.08,172130.0,31290.0,13202.0,-1.44,22.22,7.67,31.59,89.0,2.37,1.27,49.52,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.76,M-LC,29.17,103007.0,24645.0,9384.0,0.04,31.45,9.11,43.43,77.0,2.63,0.76,38.44,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,83.33,74218.0,39769.0,178.0,-0.24,115.44,0.24,115.97,NaN,223.42,0.55,13.49,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,87.81,M-MC,36.46,225272.0,310.0,167355.0,-0.89,0.14,74.29,74.53,185.0,0.00,1.66,30.79,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,29.50,H-SC,75.00,132464.0,7167.0,52045.0,2.14,5.72,39.29,47.25,139.0,0.14,0.97,65.72,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.32,H-SC,55.21,119462.0,17458.0,27560.0,-0.55,17.12,23.07,44.14,124.0,0.63,0.88,36.14,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.76,M-LC,29.17,103007.0,24645.0,9384.0,0.04,31.45,9.11,43.43,77.0,2.63,0.76,38.44,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,27.08,172130.0,31290.0,13202.0,-1.44,22.22,7.67,31.59,89.0,2.37,1.27,49.52,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,83.33,74218.0,39769.0,178.0,-0.24,115.44,0.24,115.97,NaN,223.42,0.55,13.49,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.30,M-SC,91.67,203228.0,31607.0,130472.0,-1.35,18.42,64.20,94.43,198.0,0.24,1.49,49.99,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,76.04,164827.0,13882.0,95567.0,-0.05,9.20,57.98,72.51,69.0,0.15,1.21,13.54,XR,NTT,IT
54,KPIGREEN,497.21,732.16,29.50,H-SC,75.00,132464.0,7167.0,52045.0,2.14,5.72,39.29,47.25,139.0,0.14,0.97,65.72,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-47.14,H-SC,92.71,137800.0,14464.0,119293.0,0.08,11.73,86.57,108.45,122.0,0.12,1.01,36.70,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,100.00,72134.0,-41415.0,81468.0,0.32,-36.47,112.94,35.27,261.0,-0.51,0.53,76.55,XR,NTT,HOTELS
32,ENRIN,1377.95,2975.90,NaN,NaN,83.33,74218.0,39769.0,178.0,-0.24,115.44,0.24,115.97,NaN,223.42,0.55,13.49,AR,ATH,ELECTRICAL
58,MASFIN,326.60,402.39,-19.15,H-SC,60.42,91755.0,-6225.0,28958.0,-0.71,-6.35,31.56,23.21,137.0,-0.21,0.67,33.08,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.93,X-SC,98.96,99524.0,-1439.0,95971.0,-2.26,-1.43,96.43,93.63,64.0,-0.01,0.73,48.24,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.76,M-LC,29.17,103007.0,24645.0,9384.0,0.04,31.45,9.11,43.43,77.0,2.63,0.76,38.44,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,100.00,72134.0,-41415.0,81468.0,0.32,-36.47,112.94,35.27,261.0,-0.51,0.53,76.55,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4546.37,6.93,X-SC,98.96,99524.0,-1439.0,95971.0,-2.26,-1.43,96.43,93.63,64.0,-0.01,0.73,48.24,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-215.40,H-SC,97.92,83265.0,-12051.0,12090.0,1.39,-12.64,14.52,0.04,154.0,-1.00,0.61,64.23,OX40N,NTT,MISC
52,JPPOWER,18.73,26.20,-31.42,L-SC,96.88,138283.0,-3559.0,60125.0,-0.22,-2.51,43.48,39.88,259.0,-0.06,1.02,40.14,XY24,NTT,POWER
72,SFL,1016.35,1287.00,41.99,H-SC,95.83,187476.0,-74742.0,144563.0,-1.39,-28.50,77.11,26.63,144.0,-0.52,1.38,16.56,XY24,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.97
1,25,42.00
2,50,70.44


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.50
LC    34.20
MC    21.36
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.51
X40      12.27
XY25     12.08
X40N     11.76
XR       10.14
AR        8.32
OX40N     8.16
MH        1.71
X5K       1.47
X200      1.41
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.68
M-SC    16.00
H-LC    14.22
X-LC    12.25
H-MC     9.47
X-MC     7.51
M-LC     6.66
X-SC     4.36
M-MC     4.02
L-SC     2.46
L-LC     1.07
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.16,-1.51,35.45
IT,12.10,-10.50,64.78
FINANCE,7.92,-11.69,52.12
BANKS,7.92,-5.80,48.97
MISC,6.70,-14.38,67.33
PAINTS,5.66,-13.35,35.68
ELECTRICAL,5.55,2.31,43.32
HEALTHCARE,4.52,-4.89,35.53
AUTO,4.31,-14.99,65.55


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2720314.0,27
XR,1078260.0,13
AR,1011814.0,10
X40,652982.0,9
X40N,562839.0,11
XY25,476452.0,8
OX40N,320005.0,11
SR,177598.0,2
MH,82656.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969882.0,20
M-SC,1769818.0,21
H-LC,606281.0,12
H-MC,547101.0,8
X-LC,509069.0,7
X-MC,430054.0,6
M-MC,422510.0,3
X-SC,325017.0,5
M-LC,268025.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      858369.0      6
M-SC       XY24      815323.0      7
H-SC       AR        519124.0      3
M-SC       XR        334413.0      4
M-MC       XY24      311751.0      2
H-MC       XY24      299327.0      3
X-LC       X40       241158.0      2
H-SC       X40N      230934.0      4
           XR        217977.0      3
H-LC       X40N      198715.0      4
M-SC       AR        181829.0      2
           OX40N     172938.0      5
M-LC       XY24      160371.0      3
H-LC       X40       158067.0      3
X-MC       X40       151846.0      2
L-SC       XR        143106.0      2
X-SC       XR        123412.0      1
X-LC       AR        115016.0      1
           XY25      113636.0      3
M-SC       XY25      111273.0      1
M-MC       XR        110759.0      1
X-MC       XY25      106261.0      1
           XY24       97818.0      1
X-SC       SR         95971.0      1
M-LC       XR         95567.0      1
H-SC       OX40N      91433.0      3
H-MC       XY25       88717.0      1
M-SC       SR         81627.0      1
           X40        72415.0      1
H-LC       AR         71989.0      1
           X200       68475.0      1
           X5K        68376.0      2
X-SC       XY24       65228.0      1
H-MC       AR         63920.0      1
L-SC       XY24       60125.0      1
           AR         59758.0      1
X-MC       X40N       58901.0      1
L-MC       XR         53026.0      1
H-SC       MH         52045.0      1
L-LC       XY25       44478.0      1
H-LC       XY24       40659.0      1
X-SC       OX40N      40406.0      2
X-LC       X40N       39259.0      1
H-MC       X40N       35030.0      1
           MH         30611.0      1
           X40        29496.0      1
X-MC       OX40N      15228.0      1
M-LC       XY25       12087.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
